In [1]:
#loading in the packages
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Distributions
const GRB_ENV = Gurobi.Env(output_flag=0);

In [2]:
#data frames 
swimmer_times = CSV.read("swimmer_event_times.csv",DataFrame) #column 1: the swimmer column 2: the event column 3: the time (in seconds)
swimmer_stats = CSV.read("swimmer_stats.csv",DataFrame) #column 1: the swimmer column 2: the event  column 3: the mean of all their times
#for that event column 4: the standard deviation 
individual_event_order = CSV.read("individual_event_order.csv",DataFrame) #column 1: when the event is 1-14 #column 2: what the event is
relay_event_order = CSV.read("relay_event_order.csv",DataFrame) #column 1: when the event is 1-14 #column 2: name of relay 
#column 3-6: what race the swimmer must do in each event of the relay 

Row,index,event,swimmer_1_event,swimmer_2_event,swimmer_3_event,swimmer_4_event
,Int64,String31,String7,String15,String7,String7
1,1,200 Medley Relay,50 Back,50 Breast,50 Fly,50 Free
2,14,200 Freestyle Relay,50 Free,50 Free,50 Free,50 Free


In [3]:
# Create a dictionary mapping (swimmer, event) to time
best_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_times))
# Create a dictionary for mean times
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))
# Create a dictionary for standard deviation times
std_dev_times = Dict((row[1], row[2]) => row[4] for row in eachrow(swimmer_stats))
# Create a list of individual events
individual_event_order_list = individual_event_order[:, 2]
# Create a dictionary for relay event positions
relay_event_positions = Dict(row[2] => row[3:6] for row in eachrow(relay_event_order))

Dict{String31, DataFrameRow{DataFrame, DataFrames.SubIndex{DataFrames.Index, UnitRange{Int64}, UnitRange{Int64}}}} with 2 entries:
  "200 Medley Relay"    => DataFrameRow…
  "200 Freestyle Relay" => DataFrameRow…

In [4]:
# Extract unique swimmer names from swimmer_times DataFrame
swimmer_list = unique(swimmer_times[:, 1])


31-element Vector{String31}:
 "Adler, Lauren"
 "Augustyn, Kate"
 "Beggs, Christina"
 "Bernard, Sarah"
 "Chen, Olivia"
 "Chun, Sydney"
 "Crane, Jessie"
 "Eppinger, Aria"
 "Feliz, Mary"
 "Griffin, Miri"
 "Kim, Jolie"
 "Kudela, Katie"
 "Levy, Lauren"
 ⋮
 "Shipps, Stella"
 "Simons, Kailey"
 "Smith, Sydney"
 "Swanson, Chloe"
 "Swartwood, Belise"
 "Tang, Natalie"
 "Turvey, Alexandra"
 "Yang, Iris"
 "Yang, Rachel"
 "Yao, Katherine"
 "Zhao, Kathy"
 "Zhu, Emma"

In [5]:

# Number of swimmers
swimmer_count = 31

# Number of individual events (12 events, from 2 to 13)
num_individual_events = 12

# Number of relay events (2 events: 1 and 14)
num_relay_events = 2

# Number of positions in each relay (4)
num_relay_positions = 4

# Number of groups (3 as per your formulation)
num_groups = 3

# Number of relay events
num_relay_events = nrow(relay_event_order)

penalty_time = 99999  # A large penalty value to discourage invalid assignments

99999

In [6]:
println("Number of swimmers: ", swimmer_count)
println("Number of events: ", length(individual_event_order_list))
println("Number of relay events: ", num_relay_events)

Number of swimmers: 31
Number of events: 12
Number of relay events: 2


In [7]:
println(individual_event_order_list)
println("Best times dictionary: ", best_times)


String15["1000 Free", "200 Free", "100 Back", "100 Breast", "200 Fly", "50 Free", "100 Free", "200 Back", "200 Breast", "500 Free", "100 Fly", "200 IM"]
Best times dictionary: Dict{Tuple{String31, String15}, Float64}(("Augustyn, Kate", "100 Back") => 56.85, ("Swanson, Chloe", "100 Fly") => 65.54, ("Feliz, Mary", "50 Back") => 29.35, ("Levy, Lauren", "100 Breast") => 71.74, ("Zhao, Kathy", "50 Breast") => 31.65, ("Chun, Sydney", "100 Fly") => 61.63, ("Smith, Sydney", "50 Back") => 27.47, ("Naveen, Annika", "200 Free") => 116.8, ("Bernard, Sarah", "50 Breast") => 29.97, ("Beggs, Christina", "50 Free") => 24.12, ("Lu, Sarah", "100 Free") => 58.63, ("Tang, Natalie", "500 Free") => 315.5, ("Turvey, Alexandra", "200 Free") => 116.97, ("Zhao, Kathy", "50 Free") => 25.96, ("Crane, Jessie", "400 IM") => 277.87, ("Smith, Sydney", "50 Free") => 24.85, ("Simons, Kailey", "50 Breast") => 30.34, ("Augustyn, Kate", "200 Back") => 123.96, ("Eppinger, Aria", "1000 Free") => 668.79, ("Chen, Olivia", "10

In [34]:
missing_count = 0
for s in swimmer_list
    for e in individual_event_order_list
        if !haskey(best_times, (s, e))
            best_times[(s, e)] = penalty_time  # Assign a large penalty time
            missing_count += 1
            println("Added default penalty time for swimmer $s in event $e")
        end
    end
end
println("Total missing keys: ", missing_count)


Added default penalty time for swimmer Adler, Lauren in event 1000 Free
Added default penalty time for swimmer Adler, Lauren in event 200 Free
Added default penalty time for swimmer Adler, Lauren in event 100 Back
Added default penalty time for swimmer Adler, Lauren in event 200 Fly
Added default penalty time for swimmer Adler, Lauren in event 50 Free
Added default penalty time for swimmer Adler, Lauren in event 100 Free
Added default penalty time for swimmer Adler, Lauren in event 200 Back
Added default penalty time for swimmer Adler, Lauren in event 100 Fly
Added default penalty time for swimmer Augustyn, Kate in event 1000 Free
Added default penalty time for swimmer Augustyn, Kate in event 100 Breast
Added default penalty time for swimmer Augustyn, Kate in event 200 Fly
Added default penalty time for swimmer Augustyn, Kate in event 200 Breast
Added default penalty time for swimmer Augustyn, Kate in event 100 Fly
Added default penalty time for swimmer Beggs, Christina in event 100 Br

In [8]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))
#variables 
@variable(model, x[1:swimmer_count, 1:num_individual_events], Bin) #decision variable of if swimmer competes in event 
@variable(model, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin) #decicsion variable if swimmer is 
#in the position for relay event in a certain group 

31×4×2×3 Array{VariableRef, 4}:
[:, :, 1, 1] =
 y[1,1,1,1]   y[1,2,1,1]   y[1,3,1,1]   y[1,4,1,1]
 y[2,1,1,1]   y[2,2,1,1]   y[2,3,1,1]   y[2,4,1,1]
 y[3,1,1,1]   y[3,2,1,1]   y[3,3,1,1]   y[3,4,1,1]
 y[4,1,1,1]   y[4,2,1,1]   y[4,3,1,1]   y[4,4,1,1]
 y[5,1,1,1]   y[5,2,1,1]   y[5,3,1,1]   y[5,4,1,1]
 y[6,1,1,1]   y[6,2,1,1]   y[6,3,1,1]   y[6,4,1,1]
 y[7,1,1,1]   y[7,2,1,1]   y[7,3,1,1]   y[7,4,1,1]
 y[8,1,1,1]   y[8,2,1,1]   y[8,3,1,1]   y[8,4,1,1]
 y[9,1,1,1]   y[9,2,1,1]   y[9,3,1,1]   y[9,4,1,1]
 y[10,1,1,1]  y[10,2,1,1]  y[10,3,1,1]  y[10,4,1,1]
 y[11,1,1,1]  y[11,2,1,1]  y[11,3,1,1]  y[11,4,1,1]
 y[12,1,1,1]  y[12,2,1,1]  y[12,3,1,1]  y[12,4,1,1]
 y[13,1,1,1]  y[13,2,1,1]  y[13,3,1,1]  y[13,4,1,1]
 ⋮                                      
 y[20,1,1,1]  y[20,2,1,1]  y[20,3,1,1]  y[20,4,1,1]
 y[21,1,1,1]  y[21,2,1,1]  y[21,3,1,1]  y[21,4,1,1]
 y[22,1,1,1]  y[22,2,1,1]  y[22,3,1,1]  y[22,4,1,1]
 y[23,1,1,1]  y[23,2,1,1]  y[23,3,1,1]  y[23,4,1,1]
 y[24,1,1,1]  y[24,2,1,1]  y[24,3,1,1

In [9]:
@objective(model, Min, 
    # Individual events
    sum(get(best_times, (swimmer_list[s], individual_event_order_list[e]), penalty_time) * x[s, e]
        for s in 1:swimmer_count, e in 1:num_individual_events) +
    
    # Relay events
    sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s, i, r, g]
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups)
)


99999 x[1,1] + 99999 x[1,2] + 99999 x[1,3] + 68.45 x[1,4] + 99999 x[1,5] + 99999 x[1,6] + 99999 x[1,7] + 99999 x[1,8] + 147.13 x[1,9] + 309.64 x[1,10] + 99999 x[1,11] + 131.12 x[1,12] + 99999 x[2,1] + 116.44 x[2,2] + 56.85 x[2,3] + 99999 x[2,4] + 99999 x[2,5] + 24.88 x[2,6] + 53.45 x[2,7] + 123.96 x[2,8] + 99999 x[2,9] + 305.38 x[2,10] + 99999 x[2,11] + 129.28 x[2,12] + 650.57 x[3,1] + 118.44 x[3,2] + 61.01 x[3,3] + 99999 x[3,4] + 128.45 x[3,5] + 24.12 x[3,6] + [[...1056 terms omitted...]] + 25.96 y[30,4,1,1] + 25.96 y[30,4,1,2] + 25.96 y[30,4,1,3] + 25.96 y[30,4,2,1] + 25.96 y[30,4,2,2] + 25.96 y[30,4,2,3] + 28.37 y[31,1,1,1] + 28.37 y[31,1,1,2] + 28.37 y[31,1,1,3] + 25.68 y[31,1,2,1] + 25.68 y[31,1,2,2] + 25.68 y[31,1,2,3] + 99999 y[31,2,1,1] + 99999 y[31,2,1,2] + 99999 y[31,2,1,3] + 25.68 y[31,2,2,1] + 25.68 y[31,2,2,2] + 25.68 y[31,2,2,3] + 99999 y[31,3,1,1] + 99999 y[31,3,1,2] + 99999 y[31,3,1,3] + 25.68 y[31,3,2,1] + 25.68 y[31,3,2,2] + 25.68 y[31,3,2,3] + 25.68 y[31,4,1,1] + 25.

In [10]:

# Each swimmer swims 3 individual events
@constraint(model, [s in 1:swimmer_count], 
    sum(x[s, e] for e in 1:num_individual_events) <= 3)

# Each individual event we have only one swimmer
@constraint(model, [e in 1:num_individual_events], 
    sum(x[s, e] for s in 1:swimmer_count) >= 1) 

# 4 persons per relay group
@constraint(model, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for e in 1:num_individual_events-1
        @constraint(model, x[s, e] + x[s, e+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Exclude the pairs with missing times
#@constraint(model, [s in 1:swimmer_count, e in 1:num_individual_events], 
 #   x[s, e] <= haskey(best_times, (swimmer_list[s], individual_event_order_list[e])))

# Each swimmer can only swim in one position in a group
@constraint(model, [r in 1:num_relay_events, g in 1:num_groups, i in 1:num_relay_positions], 
    sum(y[s, i, r, g] for s in 1:swimmer_count) == 1)

2×3×4 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 y[1,1,1,1] + y[2,1,1,1] + y[3,1,1,1] + y[4,1,1,1] + y[5,1,1,1] + y[6,1,1,1] + y[7,1,1,1] + y[8,1,1,1] + y[9,1,1,1] + y[10,1,1,1] + y[11,1,1,1] + y[12,1,1,1] + y[13,1,1,1] + y[14,1,1,1] + y[15,1,1,1] + y[16,1,1,1] + y[17,1,1,1] + y[18,1,1,1] + y[19,1,1,1] + y[20,1,1,1] + y[21,1,1,1] + y[22,1,1,1] + y[23,1,1,1] + y[24,1,1,1] + y[25,1,1,1] + y[26,1,1,1] + y[27,1,1,1] + y[28,1,1,1] + y[29,1,1,1] + y[30,1,1,1] + y[31,1,1,1] == 1  …  y[1,1,1,3] + y[2,1,1,3] + y[3,1,1,3] + y[4,1,1,3] + y[5,1,1,3] + y[6,1,1,3] + y[7,1,1,3] + y[8,1,1,3] + y[9,1,1,3] + y[10,1,1,3] + y[11,1,1,3] + y[12,1,1,3] + y[13,1,1,3] + y[14,1,1,3] + y[15,1,1,3] + y[16,1,1,3] + y[17,1,1,3] + y[18,1,1,3] + y[19,1,1,3] + y[20,1,1,3] + y[21,1,1,3] + y[22,1,1,3] + y[23,1,1,3] + y[24,1,1,3] + y[25,1,1,3] + y[26,1,1,3] + y[27,1,1,3] + y[28,1,1,3] + y[29

In [11]:
#optimizing 
optimize!(model)

In [12]:
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found")
else
    println("No optimal solution found")
end

Optimal solution found


In [13]:
println("Total swimming time: ", objective_value(model))

Total swimming time: 2412.7999999999997


In [14]:
#printing out what events the swimmer are swimming and what times 
for s in 1:swimmer_count
    for e in 1:num_individual_events
        if value(x[s, e]) > 0.5
            swimmer = swimmer_list[s]
            event = individual_event_order[e, :event]
            best_time = get(best_times, (swimmer, event), "N/A")
            println("Swimmer $swimmer assigned to event $event (Best time: $best_time)")
        end
    end
end

Swimmer Augustyn, Kate assigned to event 200 Back (Best time: 123.96)
Swimmer Bernard, Sarah assigned to event 200 Breast (Best time: 136.67)
Swimmer Naveen, Annika assigned to event 50 Free (Best time: 23.53)
Swimmer Roberson, Ella assigned to event 1000 Free (Best time: 624.84)
Swimmer Roberson, Ella assigned to event 500 Free (Best time: 303.66)
Swimmer Roberson, Ella assigned to event 200 IM (Best time: 125.25)
Swimmer Simons, Kailey assigned to event 100 Breast (Best time: 60.29)
Swimmer Smith, Sydney assigned to event 200 Free (Best time: 116.24)
Swimmer Smith, Sydney assigned to event 100 Free (Best time: 52.48)
Swimmer Smith, Sydney assigned to event 100 Fly (Best time: 57.21)
Swimmer Swartwood, Belise assigned to event 200 Fly (Best time: 125.71)
Swimmer Yang, Iris assigned to event 100 Back (Best time: 55.8)


In [15]:
#prints out each relay group, the position (SHOULD BE 1-4), the swimmer, and the time 
for r in 1:num_relay_events
    for g in 1:num_groups
        println("Relay $(r), Group $(g):")
        for i in 1:num_relay_positions
            for s in 1:swimmer_count
                if value(y[s, i, r, g]) > 0.5
                    swimmer = swimmer_list[s]
                    event = relay_event_order[r, "swimmer_$(i)_event"]
                    best_time = get(best_times, (swimmer, event), "N/A")
                    println("  Position $(i): $(swimmer) (Best time: $(best_time))")
                end
            end
        end
    end
end

Relay 1, Group 1:
  Position 1: Smith, Sydney (Best time: 27.47)
  Position 2: Simons, Kailey (Best time: 30.34)
  Position 3: Beggs, Christina (Best time: 20.09)
  Position 4: Chen, Olivia (Best time: 24.48)
Relay 1, Group 2:
  Position 1: Augustyn, Kate (Best time: 26.5)
  Position 2: Bernard, Sarah (Best time: 29.97)
  Position 3: Naveen, Annika (Best time: 24.89)
  Position 4: Turvey, Alexandra (Best time: 23.61)
Relay 1, Group 3:
  Position 1: Yang, Iris (Best time: 26.18)
  Position 2: Li, Anna (Best time: 30.55)
  Position 3: Zhao, Kathy (Best time: 25.44)
  Position 4: Poag, Lillian (Best time: 24.25)
Relay 2, Group 1:
  Position 1: Smith, Sydney (Best time: 24.85)
  Position 2: Chen, Olivia (Best time: 24.48)
  Position 3: Beggs, Christina (Best time: 24.12)
  Position 4: Naveen, Annika (Best time: 23.53)
Relay 2, Group 2:
  Position 1: Augustyn, Kate (Best time: 24.88)
  Position 2: Kudela, Katie (Best time: 24.87)
  Position 3: Simons, Kailey (Best time: 24.55)
  Position 4:

In [16]:
#prints out the number of events swimmers are in, both individual and relay
for s in 1:swimmer_count
    individual_events = sum(value(x[s, e]) for e in 1:num_individual_events)
    relay_events = sum(value(y[s, i, r, g]) for i in 1:num_relay_positions, r in 1:num_relay_events, g in 1:num_groups)
    total_events = individual_events + relay_events
    println("Swimmer $(swimmer_list[s]): $(total_events) events ($(individual_events) individual, $(relay_events) relay)")
end

Swimmer Adler, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Augustyn, Kate: 3.0 events (1.0 individual, 2.0 relay)
Swimmer Beggs, Christina: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Bernard, Sarah: 2.0 events (1.0 individual, 1.0 relay)
Swimmer Chen, Olivia: 2.0 events (-0.0 individual, 2.0 relay)
Swimmer Chun, Sydney: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Crane, Jessie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Eppinger, Aria: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Feliz, Mary: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Griffin, Miri: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kim, Jolie: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Kudela, Katie: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Levy, Lauren: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Li, Anna: 1.0 events (-0.0 individual, 1.0 relay)
Swimmer Lu, Sarah: -0.0 events (-0.0 individual, -0.0 relay)
Swimmer Naveen, Annika: 3.0 events (1.0 individual, 

In [2]:
using JuMP, Gurobi, CSV, DataFrames

# Load data
swimmer_times = CSV.read("swimmer_event_times.csv", DataFrame)
swimmer_stats = CSV.read("swimmer_stats.csv", DataFrame)
individual_event_order = CSV.read("individual_event_order.csv", DataFrame)
relay_event_order = CSV.read("relay_event_order.csv", DataFrame)
opponent_times = CSV.read("opponent_times.csv", DataFrame)

# Create dictionaries
best_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_times))
mean_times = Dict((row[1], row[2]) => row[3] for row in eachrow(swimmer_stats))
std_dev_times = Dict((row[1], row[2]) => row[4] for row in eachrow(swimmer_stats))
individual_event_order_list = individual_event_order[:, 2]
relay_event_positions = Dict(row[2] => row[3:6] for row in eachrow(relay_event_order))
swimmer_list = unique(swimmer_times[:, 1])
opponent_time_dict = Dict((row.event, row.swimmer) => row.time for row in eachrow(opponent_times))

# Parameters
swimmer_count = 31
num_individual_events = 12
num_relay_events = 2
num_relay_positions = 4
num_groups = 3
num_places = 6
penalty_time = 99999

# Points vectors
p = [9, 4, 3, 2, 1, 0]  # Points for individual events
P = [11, 4, 2, 0, 0, 0]  # Points for relay events

# Create model
model = Model(Gurobi.Optimizer)

# Variables
@variable(model, x[1:swimmer_count, 1:num_individual_events], Bin)
@variable(model, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model, z_isk[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model, z_iok[1:num_individual_events, 1:num_places], Bin)
@variable(model, z_gk[1:num_groups, 1:num_places], Bin)
@variable(model, S_mk[1:3, 1:num_places], Bin)

# Objective function
@objective(model, Max, 
    sum(p[k] * z_isk[s,i,k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
    sum(P[k] * z_gk[g,k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model, [s in 1:swimmer_count], 
    sum(x[s, i] for i in 1:num_individual_events) <= 3)

# 4 persons per relay group
@constraint(model, [r in 1:num_relay_events, g in 1:num_groups], 
    sum(y[s, i, r, g] for s in 1:swimmer_count, i in 1:num_relay_positions) == 4)

# Can't swim back-to-back events
for s in 1:swimmer_count
    for i in 1:num_individual_events-1
        @constraint(model, x[s, i] + x[s, i+1] <= 1)
    end
end

# Each swimmer can only swim once in first relay
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 1, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, 1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model, [s in 1:swimmer_count], 
    sum(y[s, i, 2, g] for i in 1:num_relay_positions, g in 1:num_groups) + x[s, num_individual_events] <= 1)

# Placing constraints for individual events
@constraint(model, [i in 1:num_individual_events, k in 1:5, s in 1:swimmer_count],
    z_isk[s,i,k] * get(best_times, (swimmer_list[s], individual_event_order_list[i]), penalty_time) <= 
    z_iok[i,k+1] * get(opponent_time_dict, (individual_event_order_list[i], k), penalty_time))

# Placing constraints for relay events
@constraint(model, [g in 1:num_groups, k in 1:5],
    z_gk[g,k] * sum(get(best_times, (swimmer_list[s], relay_event_positions[relay_event_order[r, 2]][i]), penalty_time) * y[s,i,r,g] 
        for s in 1:swimmer_count, i in 1:num_relay_positions, r in 1:num_relay_events) 
    <= z_gk[g,k+1] * sum(get(opponent_time_dict, (relay_event_order[r, 2], m), penalty_time) for r in 1:num_relay_events, m in 1:3))

# Only one swimmer/opponent per place per event
@constraint(model, [i in 1:num_individual_events, k in 1:num_places],
    sum(z_isk[s,i,k] for s in 1:swimmer_count) + z_iok[i,k] == 1)

# Each swimmer gets only one place per event if they swim
@constraint(model, [s in 1:swimmer_count, i in 1:num_individual_events],
    sum(z_isk[s,i,k] for k in 1:num_places) == x[s,i])

# Each relay group gets only one place
@constraint(model, [g in 1:num_groups],
    sum(z_gk[g,k] for k in 1:num_places) <= 1)

# Each opponent relay group gets only one place
@constraint(model, [m in 1:3],
    sum(S_mk[m,k] for k in 1:num_places) <= 1)

# Solve the model
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Objective value: ", objective_value(model))
    # Add code to print detailed results
else
    println("No optimal solution found")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2812 rows, 3456 columns and 12012 nonzeros
Model fingerprint: 0x08bd7264
Model has 15 quadratic constraints
Variable types: 0 continuous, 3456 integer (3456 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  QMatrix range    [2e+01, 1e+05]
  QLMatrix range   [6e+05, 6e+05]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 1913 rows and 426 columns
Presolve time: 0.10s
Presolved: 4634 rows, 6750 columns, 21717 nonzeros
Variable types: 0 continuous, 6750 integer (6750 binary)
Found heuristic solution: objective 156.0000000

Root relaxation: obj

In [3]:
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Total points: ", objective_value(model))
    
    # Individual events
    for s in 1:swimmer_count
        for i in 1:num_individual_events
            for k in 1:num_places
                if value(z_isk[s,i,k]) > 0.5
                    swimmer = swimmer_list[s]
                    event = individual_event_order_list[i]
                    place = k
                    points = p[k]
                    println("Swimmer: $swimmer, Event: $event, Place: $place, Points: $points")
                end
            end
        end
    end
    
    # Relay events
    for g in 1:num_groups
        for k in 1:num_places
            if value(z_gk[g,k]) > 0.5
                println("Relay Group $g, Place: $k, Points: $(P[k])")
                for r in 1:num_relay_events
                    for i in 1:num_relay_positions
                        for s in 1:swimmer_count
                            if value(y[s,i,r,g]) > 0.5
                                swimmer = swimmer_list[s]
                                event = relay_event_positions[relay_event_order[r, 2]][i]
                                println("  Swimmer: $swimmer, Position: $i, Event: $event")
                            end
                        end
                    end
                end
            end
        end
    end
else
    println("No optimal solution found")
end

Optimal solution found
Total points: 156.0
Swimmer: Adler, Lauren, Event: 200 Free, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 100 Breast, Place: 1, Points: 9
Swimmer: Adler, Lauren, Event: 50 Free, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Back, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 200 Fly, Place: 1, Points: 9
Swimmer: Augustyn, Kate, Event: 100 Free, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 200 Free, Place: 3, Points: 3
Swimmer: Beggs, Christina, Event: 200 Back, Place: 1, Points: 9
Swimmer: Beggs, Christina, Event: 500 Free, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 100 Back, Place: 3, Points: 3
Swimmer: Bernard, Sarah, Event: 200 Breast, Place: 1, Points: 9
Swimmer: Bernard, Sarah, Event: 100 Fly, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 1000 Free, Place: 1, Points: 9
Swimmer: Chen, Olivia, Event: 100 Breast, Place: 3, Points: 3
Swimmer: Chen, Olivia, Event: 200 IM, Place: 1, Points: 9
Swimmer: Chun, Sydney, Event

In [199]:
# 2nd model: max time
using JuMP, Gurobi

# Create the model
model = Model(Gurobi.Optimizer)

# Define parameters
swimmer_count = 31
num_individual_events = 12
num_relay_events = 2
num_relay_positions = 4
num_groups = 3
num_places = 6

# Define points vectors (example values, adjust as needed)
p = [9, 4, 3, 2, 1,0]  # Points for individual events
P = [11, 4, 2, 0, 0, 0]  # Points for relay events

# Define variables
@variable(model, x[1:swimmer_count, 1:num_individual_events], Bin)
@variable(model, y[1:swimmer_count, 1:num_relay_positions, 1:num_relay_events, 1:num_groups], Bin)
@variable(model, z_isk[1:swimmer_count, 1:num_individual_events, 1:num_places], Bin)
@variable(model, z_iok[1:num_individual_events, 1:num_places], Bin)
@variable(model, z_gk[1:num_groups, 1:num_places], Bin)
@variable(model, S_mk[1:3, 1:num_places], Bin)

# Objective function
@objective(model, Max, 
    sum(p[k] * z_isk[s,i,k] for s in 1:swimmer_count, i in 1:num_individual_events, k in 1:num_places) +
    sum(P[k] * z_gk[g,k] for g in 1:num_groups, k in 1:num_places)
)

# Constraints
# Each swimmer swims max 3 individual events
@constraint(model, [s in 1:swimmer_count], 
    sum(x[s,i] for i in 1:num_individual_events) <= 3)

# 4 persons per relay group
@constraint(model, [g in 1:num_groups], 
    sum(y[s,r,1,g] for s in 1:swimmer_count, r in 1:4) == 4)
@constraint(model, [g in 1:num_groups], 
    sum(y[s,r,2,g] for s in 1:swimmer_count, r in 5:8) == 4)

# Can't swim back-to-back events
@constraint(model, [s in 1:swimmer_count, i in 1:num_individual_events-1],
    x[s,i] + x[s,i+1] <= 1)

# Each swimmer can only swim once in first relay
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,1,g] for r in 1:4, g in 1:num_groups) <= 1)

# Each swimmer can only swim once in last relay
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,2,g] for r in 5:8, g in 1:num_groups) <= 1)

# If swim in first relay, can't swim first individual event
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,1,g] for r in 1:4, g in 1:num_groups) + x[s,1] <= 1)

# If swim in last relay, can't swim last individual event
@constraint(model, [s in 1:swimmer_count],
    sum(y[s,r,2,g] for r in 5:8, g in 1:num_groups) + x[s,num_individual_events] <= 1)

# Only one swimmer/opponent per place per event
@constraint(model, [i in 1:num_individual_events, k in 1:num_places],
    sum(z_isk[s,i,k] for s in 1:swimmer_count) + z_iok[i,k] == 1)

# Each swimmer gets only one place per event if they swim
@constraint(model, [s in 1:swimmer_count, i in 1:num_individual_events],
    sum(z_isk[s,i,k] for k in 1:num_places) == x[s,i])

# Each relay group gets only one place
@constraint(model, [g in 1:num_groups],
    sum(z_gk[g,k] for k in 1:num_places) <= 1)

# Each opponent relay group gets only one place
@constraint(model, [m in 1:3],
    sum(S_mk[m,k] for k in 1:num_places) <= 1)

# Solve the model
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found")
    println("Objective value: ", objective_value(model))
    # Add code to print detailed results
else
    println("No optimal solution found")
end


In [ ]:
# 3rd model: max points against team

In [ ]:
# 4th model: add uncertainty for against teams 